In [1]:
!pip install s3fs

    100% |████████████████████████████████| 51kB 9.7MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 12.4MB/s ta 0:00:01
    100% |████████████████████████████████| 6.5MB 3.9MB/s eta 0:00:01   3% |█▎                              | 256kB 16.1MB/s eta 0:00:01    18% |██████                          | 1.2MB 19.1MB/s eta 0:00:01    35% |███████████▍                    | 2.3MB 19.3MB/s eta 0:00:01    51% |████████████████▌               | 3.4MB 22.6MB/s eta 0:00:01    84% |███████████████████████████▏    | 5.5MB 22.9MB/s eta 0:00:01
    100% |████████████████████████████████| 1.1MB 13.6MB/s ta 0:00:01    68% |█████████████████████▉          | 757kB 24.8MB/s eta 0:00:01
  Installing build dependencies ... done
    100% |████████████████████████████████| 61kB 6.7MB/s ta 0:00:011
  Installing build dependencies ... done
    100% |████████████████████████████████| 184kB 16.1MB/s ta 0:00:01
  Installing build dependencies ... done
  Running setup.py bdist_wheel for wrapt ... don

In [6]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

In [7]:
import boto3
s3 = boto3.resource('s3',
                    region_name="us-west-2",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                   )
sampleDbBucket =  s3.Bucket("udacity-dend")

In [8]:
obj_list = list(sampleDbBucket.objects.filter(Prefix='song_data/A/B/C/'))
obj = obj_list[0]
obj

s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/B/C/TRABCAS128F14A25E2.json')

In [14]:
import pandas as pd
import numpy as np
df_song_data = pd.read_json('s3://udacity-dend/song_data/A/B/C/TRABCAS128F14A25E2.json',  lines=True)

In [10]:
obj_list = list(sampleDbBucket.objects.filter(Prefix='log_data'))
obj = obj_list[1]
obj #s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-01-events.json')
df_log_data = pd.read_json('s3://udacity-dend/log_data/2018/11/2018-11-01-events.json',  lines=True)

In [20]:
df_song_data.shape

(1, 10)

In [22]:
df_log_data.shape

(15, 18)

In [12]:
display(df_song_data.head(2), df_log_data.head(2))

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARIOZCU1187FB3A3DC,NaN,"Hamlet, NC",NaN,JOHN COLTRANE,220.44689,1,SOCEMJV12A6D4F7667,Giant Steps (Alternate Version_ Take 5_ Altern...,0


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [15]:
measurer = np.vectorize(len)

In [19]:
dict(zip(df_song_data, measurer(df_song_data.values.astype(str)).max(axis=0)))

{'artist_id': 18,
 'artist_latitude': 3,
 'artist_location': 10,
 'artist_longitude': 3,
 'artist_name': 13,
 'duration': 9,
 'num_songs': 1,
 'song_id': 18,
 'title': 50,
 'year': 1}

In [16]:
measurer(df_song_data.select_dtypes(include=[object]).values.astype(str)).max(axis=0)

array([18, 10, 13, 18, 50])

In [21]:
dict(zip(df_log_data, measurer(df_log_data.values.astype(str)).max(axis=0)))

{'artist': 26,
 'auth': 9,
 'firstName': 6,
 'gender': 1,
 'itemInSession': 1,
 'lastName': 7,
 'length': 9,
 'level': 4,
 'location': 44,
 'method': 3,
 'page': 8,
 'registration': 13,
 'sessionId': 3,
 'song': 48,
 'status': 3,
 'ts': 13,
 'userAgent': 137,
 'userId': 3}

In [24]:
df_log_data

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [2]:
import configparser


In [3]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

IAM_ROLE = config['IAM_ROLE']['ARN']
LOG_DATA = config['S3']['LOG_DATA']
LOG_JSONPATH = config['S3']['LOG_JSONPATH']
SONG_DATA = config['S3']['SONG_DATA']

In [4]:
IAM_ROLE

"'arn:aws:iam::184162717196:role/dwhRole'"